In [1]:
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras import backend as K
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from patchify import patchify, unpatchify
from tensorflow.keras import backend as K
from skimage.measure import label

def f1(y_true, y_pred, threshold=0.3):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    TP = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    Positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    Pred_Positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = TP / (Pred_Positives + tf.keras.backend.epsilon())
    recall = TP / (Positives + tf.keras.backend.epsilon())
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    
@tf.keras.utils.register_keras_serializable()
def f1_metric(y_true, y_pred):
    return f1(y_true, y_pred, threshold=0.3)


def weighted_binary_crossentropy(y_true, y_pred):
    """
    Weighted binary cross-entropy to address class imbalance.
    """
    # Define weights for foreground (root, shoot, seed) and background
    weight_foreground = 10.0
    weight_background = 1.0

    # Compute weighted binary cross-entropy
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    weights = tf.where(y_true == 1, weight_foreground, weight_background)
    loss = K.binary_crossentropy(y_true, y_pred)
    weighted_loss = loss * weights
    return K.mean(weighted_loss)
def dice_loss(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + 1e-7) / (union + 1e-7)
    return 1 - dice

@tf.keras.utils.register_keras_serializable()
def combined_loss(y_true, y_pred):
    return 0.5 * dice_loss(y_true, y_pred) + 0.5 * weighted_binary_crossentropy(y_true, y_pred)





2025-01-08 14:46:22.920011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-08 14:46:22.920086: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-08 14:46:22.921631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-08 14:46:22.930053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import logging

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("pipeline_log.txt"), logging.StreamHandler()],
)

def preprocess_image(image_path):
    """Load and preprocess the input image."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    return image

def extract_petri_dish(image):
    """Extract the largest contour assumed to be the Petri dish."""
    _, thresholded = cv2.threshold(image, 57, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        logging.warning("No contours detected for Petri dish.")
        return image, np.ones_like(image)

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

def predict_root_mask(image, model, patch_size=128, stride=64):
    """Predict root mask using a patch-based model."""
    h, w = image.shape
    patches = []
    positions = []
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size]
            patch_rgb = np.stack([patch]*3, axis=-1)
            patches.append(patch_rgb)
            positions.append((y, x))

    patches = np.array(patches) / 255.0
    predictions = model.predict(patches, verbose=0)

    reconstructed = np.zeros((h, w), dtype=np.float32)
    counts = np.zeros((h, w), dtype=np.float32)
    for pred, (y, x) in zip(predictions, positions):
        pred = pred[..., 0]
        reconstructed[y:y+patch_size, x:x+patch_size] += pred
        counts[y:y+patch_size, x:x+patch_size] += 1
    return (reconstructed / np.maximum(counts, 1) > 0.3).astype(np.uint8)

def connect_roots(mask):
    """Dilate the mask to connect fragmented roots."""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    return cv2.dilate(mask, kernel, iterations=27)

def filter_and_select_largest_objects(mask, min_area=500, max_objects=5):
    """Filter and retain the largest root components."""
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    valid_objects = [(i, stats[i, cv2.CC_STAT_AREA]) for i in range(1, num_labels) if stats[i, cv2.CC_STAT_AREA] >= min_area]
    largest_objects = sorted(valid_objects, key=lambda x: x[1], reverse=True)[:max_objects]

    filtered_mask = np.zeros_like(mask, dtype=np.uint8)
    for obj_id, _ in largest_objects:
        filtered_mask[labels == obj_id] = 255
    return filtered_mask, largest_objects

def measure_bounding_boxes(filtered_mask):
    """Measure roots with bounding boxes."""
    contours, _ = cv2.findContours(filtered_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
    bounding_boxes = sorted(bounding_boxes, key=lambda b: b[0])
    return bounding_boxes

def find_lowest_point(mask, bounding_box):
    """Find the lowest point in a bounding box."""
    x, y, w, h = bounding_box
    roi = mask[y:y+h, x:x+w]
    coordinates = np.column_stack(np.where(roi > 0))
    if len(coordinates) == 0:
        return None
    lowest_local = coordinates[np.argmax(coordinates[:, 0])]
    return lowest_local + [y, x]

def convert_to_mm(pixel_coords, image_shape, plate_size_mm=150):
    """Convert pixel coordinates to mm-space."""
    h, w = image_shape
    conversion_factor_x = plate_size_mm / w
    conversion_factor_y = plate_size_mm / h
    return np.array([pixel_coords[1] * conversion_factor_x, pixel_coords[0] * conversion_factor_y])

# Directory containing images
input_dir = "kaggle_test"
model_path = "232430_unet_model_128px_v7md.keras"
plate_position_robot = np.array([0.10775, 0.088 - 0.026, 0.057])  # Adjust based on your setup

# Load the model
model = tf.keras.models.load_model(model_path)

# Process all images in the directory
all_root_tip_coords = {}

for image_name in sorted(os.listdir(input_dir)):
    if not image_name.endswith((".png", ".jpg", ".jpeg")):
        continue

    image_path = os.path.join(input_dir, image_name)
    logging.info(f"Processing image: {image_name}")

    try:
        # Process the image
        image = preprocess_image(image_path)
        petri_dish = extract_petri_dish(image)
        predicted_mask = predict_root_mask(petri_dish, model)
        connected_mask = connect_roots(predicted_mask)

        # Filter and select largest components
        filtered_mask, _ = filter_and_select_largest_objects(connected_mask, min_area=500, max_objects=5)
        bounding_boxes = measure_bounding_boxes(filtered_mask)

        # Extract coordinates in robot space
        root_tip_coords = []
        for box in bounding_boxes:
            point = find_lowest_point(filtered_mask, box)
            if point is not None:
                mm_coords = convert_to_mm(point, petri_dish.shape)
                robot_coords = np.append(mm_coords, 0.057) + plate_position_robot
                root_tip_coords.append(robot_coords)

        all_root_tip_coords[image_name] = root_tip_coords
        logging.info(f"Extracted Root Tip Coordinates for {image_name}: {root_tip_coords}")

    except Exception as e:
        logging.error(f"Error processing image {image_name}: {e}")

# Log all extracted root tip coordinates
logging.info("All Extracted Root Tip Coordinates:")
for img_name, coords in all_root_tip_coords.items():
    logging.info(f"{img_name}: {coords}")


2025-01-08 14:46:27.903936: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-08 14:46:27.904414: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15068 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c1:00.0, compute capability: 8.6
INFO: Processing image: test_image_1.png
2025-01-08 14:46:34.978449: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
INFO: Extracted Root Tip Coordinates for test_image_1.png: [array([ 6.89359458, 43.1754469 ,  0.114     ]), array([16.52511592, 62.63902066,  0.114     ]), array([23.2015114 , 54.59262704,  0.114     ]), array([76.12015423, 39.47845524,  0.114     ]), array([1.25864773e+02, 4.50239427e+01, 1.14000000e-01])]
INFO: Processing image: test_image_10.png
INFO: Extracted Roo

In [3]:
import numpy as np
import logging
from pid_class import PIDController
from ot2_class import OT2Env

# Logging configuration
log_file = "adaptive_pid_optimized_ot2.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler(log_file)],
)

logging.info("Starting Optimized PID Controller Test with OT2Env")

# Initialize the OT2 environment
env = OT2Env()

# Initialize PID parameters
pid_x = PIDController(Kp=10, Ki=0.1, Kd=2, dt=0.5, integral_limit=5.0)
pid_y = PIDController(Kp=10, Ki=0.1, Kd=2, dt=0.5, integral_limit=5.0)
pid_z = PIDController(Kp=10, Ki=0.1, Kd=2, dt=0.5, integral_limit=1.0)

# Example root tip coordinates (replace with actual coordinates)
root_tip_coords = {
    "image_1.png": [[0.12, 0.05, 0.057], [0.15, 0.08, 0.057]],
    "image_2.png": [[0.11, 0.07, 0.057], [0.14, 0.09, 0.057]],
}

if not root_tip_coords:
    logging.error("No root tip coordinates provided.")
    print("No root tip coordinates found. Exiting...")
    exit()

for image_name, coordinates in root_tip_coords.items():
    logging.info(f"Processing root tips for {image_name}")
    print(f"Processing {image_name}...")

    for idx, target_position in enumerate(coordinates):
        logging.info(f"Root Tip {idx + 1}: Target Position: {target_position}")
        print(f"Root Tip {idx + 1}: Target Position: {target_position}")

        try:
            observation, info = env.reset()
            print(f"Environment reset. Initial observation: {observation}")
        except Exception as e:
            logging.error(f"Environment reset failed: {e}")
            print(f"Failed to reset environment for {image_name}, Root Tip {idx + 1}.")
            continue

        # Initialize loop variables
        terminated = False
        epoch = 0
        all_error = 0
        recent_errors = []

        # Simulation loop
        while not terminated and epoch < 200:
            epoch += 1

            # Current pipette position
            current_position = observation[:3]

            # Compute individual axis errors
            error_x = abs(target_position[0] - current_position[0])
            error_y = abs(target_position[1] - current_position[1])
            error_z = abs(target_position[2] - current_position[2])
            error = np.linalg.norm(target_position - current_position)
            all_error += error

            # Log errors and coordinates
            logging.info(f"Epoch: {epoch}")
            logging.info(f"Current Position: X={current_position[0]:.4f}, Y={current_position[1]:.4f}, Z={current_position[2]:.4f}")
            logging.info(f"Errors: X={error_x:.4f}, Y={error_y:.4f}, Z={error_z:.4f}")
            logging.info(f"Euclidean Error: {error:.4f}")
            print(f"Epoch {epoch}: Euclidean Error = {error:.4f}")

            # Success criterion
            if error <= 0.005:
                logging.info(f"Root Tip {idx + 1}: Target reached successfully in {epoch} epochs. Final Error: {error:.4f}")
                print(f"Root Tip {idx + 1} reached in {epoch} epochs with final error {error:.4f}")
                break

            # Adjust PID parameters dynamically
            if error > 0.1:
                pid_x.Ki, pid_x.Kd = 0.20, 3
                pid_y.Ki, pid_y.Kd = 0.20, 3
                pid_z.Ki, pid_z.Kd = 0.18, 2.8
            elif 0.05 < error <= 0.1:
                pid_x.Ki, pid_x.Kd = 0.12, 1.8
                pid_y.Ki, pid_y.Kd = 0.12, 1.8
                pid_z.Ki, pid_z.Kd = 0.1, 1.5
            else:
                pid_x.Ki, pid_x.Kd = 0.1, 1.2
                pid_y.Ki, pid_y.Kd = 0.1, 1.2
                pid_z.Ki, pid_z.Kd = 0.08, 1.0

            # Compute control actions with action saturation
            control_x = np.clip(pid_x.compute(current_position[0]), -1, 1)
            control_y = np.clip(pid_y.compute(current_position[1]), -1, 1)
            control_z = np.clip(pid_z.compute(current_position[2]), -1, 1)
            action = np.array([control_x, control_y, control_z])

            # Take a step in the environment
            try:
                observation, _, terminated, truncated, _ = env.step(action)
            except Exception as e:
                logging.error(f"Environment step failed: {e}")
                print(f"Failed to take a step for {image_name}, Root Tip {idx + 1}.")
                break

            # Track recent errors for early termination
            recent_errors.append(error)
            if len(recent_errors) > 20:
                recent_errors.pop(0)
                avg_error_change = abs(recent_errors[-1] - recent_errors[0]) / 5
                if avg_error_change < 0.0005:
                    logging.info(f"Terminating early due to negligible error change after {epoch} epochs.")
                    print(f"Early termination at epoch {epoch} due to negligible error change.")
                    break

            # Handle truncation
            if truncated:
                logging.warning("Environment truncated. Resetting...")
                observation, info = env.reset()
                pid_x.setpoint = target_position[0]
                pid_y.setpoint = target_position[1]
                pid_z.setpoint = target_position[2]

        # Log final result for this root tip
        if error > 0.005:
            logging.warning(f"Root Tip {idx + 1}: Failed to reach target within 200 epochs.")
            print(f"Root Tip {idx + 1} failed to reach target within 200 epochs.")

logging.info("PID Controller Test Complete.")
print("PID Controller Test Complete. Check the log file for details.")


INFO: Starting Optimized PID Controller Test with OT2Env
pybullet build time: Nov 28 2023 23:48:36
INFO: Processing root tips for image_1.png
INFO: Root Tip 1: Target Position: [0.12, 0.05, 0.057]
INFO: Epoch: 1
INFO: Current Position: X=0.0730, Y=0.0895, Z=0.1195
INFO: Errors: X=0.0470, Y=0.0395, Z=0.0625
INFO: Euclidean Error: 0.0876


Current working directory: /home/y2b/renforcement_learning_232430
Processing image_1.png...
Root Tip 1: Target Position: [0.12, 0.05, 0.057]
Environment reset. Initial observation: [ 0.073       0.0895      0.1195     -0.28448522  0.19613717 -0.0504676 ]
Epoch 1: Euclidean Error = 0.0876


AttributeError: 'PIDController' object has no attribute 'setpoint'